# Module 16: Week 5 Analysis - Advanced Neural Network Optimization

This notebook documents the strategy, analysis, and query generation for **Week 5** (Module 16).

## Focus
- **Deep Learning Concepts**: Applying insights from AlexNet, ImageNet, and modern architectures
- **Enhanced Neural Network Surrogates**: Building on Week 4's NN approach with architectural improvements
- **Framework Parallels**: Drawing connections between PyTorch/TensorFlow design and BBO strategy
- **Feature Hierarchies**: Understanding how learned representations guide optimization

---

## Module 16 Concepts Applied to BBO

### Key Deep Learning Insights

**1. AlexNet's Breakthrough (2012)**
- Demonstrated that **depth + data + compute** can solve complex problems
- Dropout regularization prevents overfitting with limited data
- ReLU activations enable efficient gradient flow

**Application to BBO**: Our ensemble of MLPs uses dropout and ReLU, inspired by AlexNet's success.

**2. Five Building Blocks of Deep Learning**
1. **Linear layers**: Transform input dimensions
2. **Activations**: Introduce non-linearity (ReLU, sigmoid)
3. **Normalization**: Stabilize training (BatchNorm, LayerNorm)
4. **Regularization**: Prevent overfitting (dropout, weight decay)
5. **Loss functions**: Define the optimization objective

**Application to BBO**: All five blocks are present in our surrogate architecture.

**3. Framework Choice (PyTorch vs TensorFlow)**
- PyTorch: Dynamic graphs, easier debugging, research-friendly
- TensorFlow: Production-ready, distributed training, deployment

**Application to BBO**: We use PyTorch for its dynamic gradient computation, essential for computing ∂y/∂x at arbitrary points.

**4. Feature Hierarchies**
- Early layers: Detect simple patterns (edges, textures)
- Deeper layers: Combine into complex features
- Final layers: Task-specific representations

**Application to BBO**: Our 2-layer MLP learns which input dimensions matter (feature hierarchy) for predicting function output.

---

In [ ]:
import sys
import os

# Add the src directory to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import load_data, save_submission
import warnings

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# sklearn for preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C

# Configuration
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
%matplotlib inline

# Set seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("Week 5 BBO Analysis - Module 16: Advanced Neural Networks - Setup Complete")

## Week 4 Results Analysis

| Function | Week 3 | Week 4 | Best Ever | Change | Notes |
|----------|--------|--------|-----------|--------|-------|
| 1 | ~0 | **0.196** | **0.196 (W4)** | 🚀 | **BREAKTHROUGH!** First non-zero value! |
| 2 | 0.345 | **0.667** | **0.667 (W4)** | 🚀 | **NEW BEST!** Beat initial (0.611) |
| 3 | -0.041 | -0.085 | -0.035 (init) | ↘ | Slight regression |
| 4 | -1.27 | -24.52 | 0.600 (W1) | ↘ | Exploration failed |
| 5 | 1509.0 | 1289.3 | 1618.5 (W1) | ↘ | Below best |
| 6 | -0.769 | **-0.717** | **-0.714 (init)** | ↗ | Near best! |
| 7 | 2.229 | **2.396** | **2.396 (W4)** | 🚀 | **NEW BEST!** |
| 8 | 9.915 | 9.877 | 9.915 (W3) | ↘ | Slight regression |

### Key Insights for Week 5

**Major Successes (🚀)**:
- **F1**: Grid search at [0.65, 0.65] found the first non-zero value (0.196)! This is huge.
- **F2**: Returning to best initial point yielded 0.667 - even better than initial 0.611!
- **F7**: Micro-perturbation improved from 2.29 to 2.396 - new global best!

**Lessons Learned**:
- **F4 Exploration Failed**: Moving to [0.75, 0.75, 0.75, 0.75] gave -24.52. The positive region is NOT in the high corner.
- **F5 Gradient Over-corrected**: Moving away from Week 1's best reduced performance.
- **F6 Near Optimal**: Returning to initial gave -0.717, very close to best (-0.714).

---

In [ ]:
class EnhancedSurrogateNN(nn.Module):
    """Enhanced Neural Network surrogate with Module 16 concepts.
    
    Improvements inspired by AlexNet and modern architectures:
    - Deeper network option for complex functions
    - Skip connections for gradient flow (ResNet-inspired)
    - Better initialization (Xavier/He)
    """
    
    def __init__(self, input_dim, hidden_dims=[64, 32], dropout=0.1, use_skip=False):
        super(EnhancedSurrogateNN, self).__init__()
        
        self.use_skip = use_skip and (input_dim == hidden_dims[-1] if hidden_dims else False)
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),  # Stabilizes training
                nn.ReLU(),                  # AlexNet's key activation
                nn.Dropout(dropout)         # Regularization
            ])
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
        
        # Xavier initialization (better for ReLU)
        self._init_weights()
    
    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    def forward(self, x):
        return self.network(x).squeeze(-1)


class ImprovedEnsembleSurrogate:
    """Ensemble of NNs with improved training inspired by Module 16.
    
    Key improvements:
    1. Cosine annealing LR schedule (like modern vision models)
    2. Gradient clipping for stability
    3. Early stopping with patience
    4. Diverse architectures in ensemble
    """
    
    def __init__(self, input_dim, n_samples, n_models=5):
        self.input_dim = input_dim
        self.n_models = n_models
        self.models = []
        self.optimizers = []
        self.schedulers = []
        self.scalers = {'X': StandardScaler(), 'y': StandardScaler()}
        
        # Adaptive architecture based on data size
        base_hidden = max(16, min(64, n_samples * 2))
        
        # Diverse architectures for ensemble (like ensemble methods in competitions)
        architectures = [
            [base_hidden, base_hidden // 2],                    
            [base_hidden * 2, base_hidden, base_hidden // 2],   
            [base_hidden, base_hidden],                          
            [base_hidden // 2, base_hidden // 4],               
            [base_hidden, base_hidden, base_hidden // 2],       
        ]
        
        dropout = min(0.3, max(0.1, 1.0 / np.sqrt(n_samples)))
        
        for i in range(n_models):
            torch.manual_seed(42 + i * 7)
            arch = architectures[i % len(architectures)]
            model_dropout = dropout * (0.8 + 0.4 * np.random.random())
            
            model = EnhancedSurrogateNN(input_dim, arch, model_dropout).to(device)
            
            lr = 0.01 / (1 + len(arch) * 0.2)
            weight_decay = 1e-3 / n_samples
            
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200, eta_min=1e-5)
            
            self.models.append(model)
            self.optimizers.append(optimizer)
            self.schedulers.append(scheduler)
    
    def fit(self, X, y, max_epochs=1000, patience=100, verbose=False):
        X_scaled = self.scalers['X'].fit_transform(X)
        y_scaled = self.scalers['y'].fit_transform(y.reshape(-1, 1)).flatten()
        
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        y_tensor = torch.FloatTensor(y_scaled).to(device)
        
        criterion = nn.MSELoss()
        
        for model_idx, (model, optimizer, scheduler) in enumerate(
            zip(self.models, self.optimizers, self.schedulers)
        ):
            model.train()
            best_loss = float('inf')
            patience_counter = 0
            best_state = None
            
            for epoch in range(max_epochs):
                optimizer.zero_grad()
                pred = model(X_tensor)
                loss = criterion(pred, y_tensor)
                loss.backward()
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                scheduler.step()
                
                current_loss = loss.item()
                
                if current_loss < best_loss - 1e-6:
                    best_loss = current_loss
                    patience_counter = 0
                    best_state = model.state_dict().copy()
                else:
                    patience_counter += 1
                
                if patience_counter >= patience:
                    if verbose:
                        print(f"  Model {model_idx}: Early stop at epoch {epoch}, loss={best_loss:.6f}")
                    break
            
            if best_state is not None:
                model.load_state_dict(best_state)
    
    def predict(self, X, return_std=True):
        X_scaled = self.scalers['X'].transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        
        predictions = []
        for model in self.models:
            model.eval()
            with torch.no_grad():
                pred_scaled = model(X_tensor).cpu().numpy()
                pred = self.scalers['y'].inverse_transform(pred_scaled.reshape(-1, 1)).flatten()
                predictions.append(pred)
        
        predictions = np.array(predictions)
        mean = predictions.mean(axis=0)
        std = predictions.std(axis=0)
        
        if return_std:
            return mean, std
        return mean
    
    def gradient_at(self, x):
        x_scaled = self.scalers['X'].transform(x.reshape(1, -1))
        x_tensor = torch.FloatTensor(x_scaled).to(device).requires_grad_(True)
        
        gradients = []
        for model in self.models:
            model.eval()
            pred = model(x_tensor)
            pred.backward()
            grad = x_tensor.grad.cpu().numpy().flatten()
            grad = grad / self.scalers['X'].scale_
            gradients.append(grad)
            x_tensor.grad.zero_()
        
        return np.mean(gradients, axis=0)


print("Enhanced Neural Network Surrogate Classes Defined")
print("  - Kaiming initialization (He et al.)")
print("  - Cosine annealing LR schedule")
print("  - Gradient clipping for stability")
print("  - Diverse ensemble architectures")

In [ ]:
def nn_ucb_acquisition(X, ensemble, kappa=1.96):
    """Upper Confidence Bound using NN ensemble."""
    mu, sigma = ensemble.predict(X, return_std=True)
    return mu + kappa * sigma


def nn_expected_improvement(X, ensemble, y_best, xi=0.01):
    """Expected Improvement using NN ensemble."""
    from scipy.stats import norm
    
    mu, sigma = ensemble.predict(X, return_std=True)
    
    with np.errstate(divide='warn'):
        imp = mu - y_best - xi
        Z = np.divide(imp, sigma, out=np.zeros_like(imp), where=sigma > 1e-6)
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma < 1e-6] = 0.0
    
    return ei


def gradient_ascent_acquisition(ensemble, start_x, bounds, n_steps=100, lr=0.01):
    """Gradient ascent on the NN surrogate."""
    x = start_x.copy()
    lower = np.array([b[0] for b in bounds])
    upper = np.array([b[1] for b in bounds])
    
    for step in range(n_steps):
        grad = ensemble.gradient_at(x)
        x = x + lr * grad
        x = np.clip(x, lower, upper)
    
    return x


def local_perturbation(best_x, bounds, perturbation_scale=0.05, bias=None):
    """Generate a point by perturbing the best known point."""
    dim = len(best_x)
    lower = np.array([b[0] for b in bounds])
    upper = np.array([b[1] for b in bounds])
    ranges = upper - lower
    
    noise = np.random.normal(0, perturbation_scale, dim) * ranges
    new_x = best_x + noise
    
    if bias is not None:
        new_x = new_x + np.array(bias)
    
    return np.clip(new_x, lower, upper)


def format_query(x):
    """Formats the query point as a string for submission."""
    return "-".join([f"{val:.6f}" for val in x])


print("Acquisition Functions Defined")

In [ ]:
from IPython.display import display, Markdown

# Load and analyze all functions with Week 4 results
function_data = {}

print("="*80)
print("DATA SUMMARY - Week 5 (14 data points per 2D function)")
print("="*80)

for func_id in range(1, 9):
    df = load_data(func_id)
    input_cols = [c for c in df.columns if c.startswith('x')]
    X = df[input_cols].values
    y = df['y'].values
    
    function_data[func_id] = {
        'df': df,
        'X': X,
        'y': y,
        'dim': len(input_cols),
        'y_best': y.max(),
        'best_idx': y.argmax(),
        'best_x': X[y.argmax()]
    }
    
    # Find best source
    best_source = df.iloc[y.argmax()]['source']
    
    display(Markdown(f"### Function {func_id} ({len(input_cols)}D)"))
    print(f"Samples: {len(df)} | Output range: [{y.min():.4f}, {y.max():.4f}]")
    print(f"Best: y={y.max():.6f} at {format_query(X[y.argmax()])} ({best_source})")
    
    # Show Week 4 result
    week4_data = df[df['source'] == 'week_4_submission']
    if len(week4_data) > 0:
        w4_y = week4_data['y'].values[0]
        status = "✓ NEW BEST" if w4_y == y.max() else f"(best: {y.max():.4f})"
        print(f"Week 4 result: y = {w4_y:.6f} {status}")
    print()

In [ ]:
# Train improved NN ensembles for each function
print("Training Enhanced Neural Network Ensembles (Module 16 Architecture)...")
print("=" * 60)

ensembles = {}

for func_id in range(1, 9):
    data = function_data[func_id]
    dim = data['dim']
    X = data['X']
    y = data['y']
    n_samples = len(X)
    
    print(f"\nFunction {func_id} ({dim}D, {n_samples} samples):")
    
    ensemble = ImprovedEnsembleSurrogate(
        input_dim=dim,
        n_samples=n_samples,
        n_models=5
    )
    
    ensemble.fit(X, y, max_epochs=1000, patience=100, verbose=False)
    
    ensembles[func_id] = ensemble
    
    # Evaluate ensemble quality
    y_pred, y_std = ensemble.predict(X)
    mse = np.mean((y_pred - y) ** 2)
    r2 = 1 - mse / np.var(y)
    print(f"  Training R²: {r2:.4f} | Mean uncertainty: {y_std.mean():.4f}")

print("\n" + "=" * 60)
print("All enhanced ensembles trained successfully!")

## Week 5 Strategy: Evidence-Based Refinement

### Week 4 Outcomes Analysis

| Function | W4 Strategy | W4 Result | W4 vs Best | Week 5 Action |
|----------|-------------|-----------|------------|---------------|
| **F1** | Grid search | 0.196 | **NEW BEST** 🚀 | Exploit around [0.65, 0.65] |
| **F2** | Exact return | 0.667 | **NEW BEST** 🚀 | Micro-perturb around [0.70, 0.93] |
| F3 | NN gradient | -0.085 | Worse | Return to initial best |
| F4 | Exploration | -24.52 | Much worse | Return to W1 coordinates |
| F5 | NN gradient | 1289.3 | Worse | Return to W1 coordinates |
| **F6** | Exact return | -0.717 | Very close | Tiny perturbation |
| **F7** | Micro-perturb | 2.396 | **NEW BEST** 🚀 | Continue micro-perturb |
| F8 | NN gradient | 9.877 | Close | Return to W3 best |

### Key Strategic Insights

1. **F1 Breakthrough**: The [0.65, 0.65] region is special - explore nearby!
2. **Exact Return Works**: F2, F6 benefited from returning to known good points
3. **NN Gradients Over-corrected**: F3, F5, F8 all moved away from optima
4. **F4 Exploration Failed Badly**: The positive region is very localized

### Week 5 Strategy Summary

| Function | Strategy | Rationale |
|----------|----------|----------|
| 1 | **EXPLOIT_DISCOVERY** | Found 0.196 at [0.65, 0.65] - search nearby |
| 2 | **MICRO_PERTURB** | W4 beat all previous - tiny refinement |
| 3 | **EXACT_RETURN** | Return to initial best (-0.035) |
| 4 | **EXACT_RETURN** | Must return to W1's only positive value |
| 5 | **EXACT_RETURN** | Return to W1's 1618.5 peak |
| 6 | **MICRO_PERTURB** | Very close to best, tiny refinement |
| 7 | **GRADIENT_REFINE** | W4 success - continue gradient approach |
| 8 | **EXACT_RETURN** | Return to W3's 9.915 peak |

---

In [ ]:
# Special analysis for F1 - the breakthrough function
print("="*60)
print("SPECIAL ANALYSIS: Function 1 - Exploiting the Breakthrough")
print("="*60)

data = function_data[1]
ensemble = ensembles[1]
X = data['X']
y = data['y']

# Find the breakthrough point
best_idx = y.argmax()
best_x = X[best_idx]
best_y = y[best_idx]

print(f"\nBreakthrough point: {format_query(best_x)}")
print(f"Value: {best_y:.6f}")

# Analyze gradient at breakthrough point
grad = ensemble.gradient_at(best_x)
print(f"\nGradient at breakthrough: [{grad[0]:.4f}, {grad[1]:.4f}]")
print(f"Gradient magnitude: {np.linalg.norm(grad):.4f}")
print(f"Gradient direction suggests: move x0 {'up' if grad[0] > 0 else 'down'}, x1 {'up' if grad[1] > 0 else 'down'}")

# Visualize the neighborhood
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: NN prediction surface around breakthrough
res = 40
x1_grid = np.linspace(0.5, 0.8, res)
x2_grid = np.linspace(0.5, 0.8, res)
X1, X2 = np.meshgrid(x1_grid, x2_grid)
X_grid = np.column_stack([X1.ravel(), X2.ravel()])

mu_nn, sigma_nn = ensemble.predict(X_grid)

im1 = axes[0].contourf(X1, X2, mu_nn.reshape(res, res), cmap='viridis', levels=20)
axes[0].scatter([best_x[0]], [best_x[1]], c='red', s=200, marker='*', edgecolors='white', zorder=5)
axes[0].set_title('NN Predicted Mean (zoomed)')
axes[0].set_xlabel('x0')
axes[0].set_ylabel('x1')
plt.colorbar(im1, ax=axes[0])

# Plot 2: Uncertainty
im2 = axes[1].contourf(X1, X2, sigma_nn.reshape(res, res), cmap='plasma', levels=20)
axes[1].scatter([best_x[0]], [best_x[1]], c='red', s=200, marker='*', edgecolors='white', zorder=5)
axes[1].set_title('NN Uncertainty (zoomed)')
axes[1].set_xlabel('x0')
axes[1].set_ylabel('x1')
plt.colorbar(im2, ax=axes[1])

# Plot 3: Sample candidate points
np.random.seed(42)
candidates = []
for _ in range(100):
    perturbation = np.random.normal(0, 0.05, 2)
    candidate = np.clip(best_x + perturbation, 0.01, 0.99)
    candidates.append(candidate)
candidates = np.array(candidates)

mu_cand, sigma_cand = ensemble.predict(candidates)
ucb_cand = mu_cand + 2.0 * sigma_cand

im3 = axes[2].scatter(candidates[:, 0], candidates[:, 1], c=ucb_cand, cmap='coolwarm', s=50)
axes[2].scatter([best_x[0]], [best_x[1]], c='black', s=200, marker='*', edgecolors='white', zorder=5)
axes[2].set_title('UCB at Candidate Points')
axes[2].set_xlabel('x0')
axes[2].set_ylabel('x1')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

# Find best candidate
best_cand_idx = ucb_cand.argmax()
print(f"\nBest candidate point: {format_query(candidates[best_cand_idx])}")
print(f"UCB value: {ucb_cand[best_cand_idx]:.4f}")

In [ ]:
# Week 5 Configuration based on Week 4 outcomes
WEEK5_CONFIG = {
    1: {
        'strategy': 'EXPLOIT_DISCOVERY',
        'center': [0.65, 0.65],  # Week 4 breakthrough point
        'perturb_scale': 0.03,   # Small perturbation around success
        'desc': '2D - Exploit breakthrough region (0.196 found at [0.65, 0.65])'
    },
    2: {
        'strategy': 'MICRO_PERTURB',
        'target': [0.702637, 0.926564],  # Week 4 best (0.667)
        'perturb_scale': 0.02,
        'desc': '2D - Micro-perturb W4 best (0.667 beat all previous)'
    },
    3: {
        'strategy': 'EXACT_RETURN',
        'target': [0.492581, 0.611593, 0.340176],  # Initial best (-0.035)
        'desc': '3D - Return to initial best (-0.035), W4 regression'
    },
    4: {
        'strategy': 'EXACT_RETURN',
        'target': [0.404559, 0.414786, 0.357365, 0.399048],  # W1 only positive
        'desc': '4D - Return to W1 coordinates (only positive value)'
    },
    5: {
        'strategy': 'EXACT_RETURN',
        'target': [0.362718, 0.273413, 0.996088, 0.997538],  # W1 peak
        'desc': '4D - Return to W1 peak (1618.5)'
    },
    6: {
        'strategy': 'MICRO_PERTURB',
        'target': [0.728186, 0.154693, 0.732552, 0.693997, 0.056401],  # W4 result
        'perturb_scale': 0.01,
        'desc': '5D - Micro-perturb (W4 very close to best)'
    },
    7: {
        'strategy': 'NN_GRADIENT',
        'start': [0.037482, 0.151519, 0.584151, 0.238672, 0.376091, 0.784901],  # W4 best
        'lr': 0.005,  # Conservative learning rate
        'desc': '6D - Gradient refinement from W4 best (2.396)'
    },
    8: {
        'strategy': 'EXACT_RETURN',
        'target': [0.024511, 0.095108, 0.162460, 0.036406, 0.886768, 0.318315, 0.166845, 0.204731],  # W3 best
        'desc': '8D - Return to W3 best (9.915)'
    }
}

# Generate Week 5 queries
week5_queries = {}

print("=" * 80)
print("WEEK 5 QUERY GENERATION - Evidence-Based Strategy")
print("=" * 80)

for func_id, config in WEEK5_CONFIG.items():
    display(Markdown(f"### Function {func_id}: {config['desc']}"))
    
    data = function_data[func_id]
    ensemble = ensembles[func_id]
    dim = data['dim']
    bounds = [(0.01, 0.99)] * dim
    y_best = data['y_best']
    
    strategy = config['strategy']
    np.random.seed(42 + func_id)  # Reproducible randomness
    
    if strategy == 'EXACT_RETURN':
        next_x = np.array(config['target'])
        print(f"Strategy: EXACT RETURN to proven coordinates")
        pred_mu, pred_std = ensemble.predict(next_x.reshape(1, -1))
        print(f"NN prediction at target: y = {pred_mu[0]:.4f} ± {pred_std[0]:.4f}")
        
    elif strategy == 'EXPLOIT_DISCOVERY':
        center = np.array(config['center'])
        scale = config['perturb_scale']
        
        # Use gradient to inform direction
        grad = ensemble.gradient_at(center)
        grad_normalized = grad / (np.linalg.norm(grad) + 1e-8)
        
        # Generate candidates around center, biased by gradient
        best_ucb = -np.inf
        best_candidate = center
        
        for _ in range(100):
            noise = np.random.normal(0, scale, dim)
            # Add small gradient bias
            candidate = center + noise + 0.01 * grad_normalized
            candidate = np.clip(candidate, 0.01, 0.99)
            
            ucb = nn_ucb_acquisition(candidate.reshape(1, -1), ensemble, kappa=2.0)[0]
            if ucb > best_ucb:
                best_ucb = ucb
                best_candidate = candidate
        
        next_x = best_candidate
        print(f"Strategy: EXPLOIT DISCOVERY around breakthrough")
        print(f"Gradient direction: [{grad[0]:.4f}, {grad[1]:.4f}]")
        pred_mu, pred_std = ensemble.predict(next_x.reshape(1, -1))
        print(f"Best candidate UCB: {best_ucb:.4f}, predicted: {pred_mu[0]:.4f}")
        
    elif strategy == 'MICRO_PERTURB':
        target = np.array(config['target'])
        scale = config.get('perturb_scale', 0.02)
        
        # Use gradient to inform perturbation direction
        grad = ensemble.gradient_at(target)
        grad_normalized = grad / (np.linalg.norm(grad) + 1e-8)
        
        noise = np.random.normal(0, scale, dim)
        next_x = target + noise + 0.005 * grad_normalized
        next_x = np.clip(next_x, 0.01, 0.99)
        
        print(f"Strategy: MICRO PERTURBATION (scale={scale})")
        print(f"Gradient direction: {['%.4f' % g for g in grad]}")
        pred_mu, pred_std = ensemble.predict(next_x.reshape(1, -1))
        print(f"Predicted: y = {pred_mu[0]:.4f} ± {pred_std[0]:.4f}")
        
    elif strategy == 'NN_GRADIENT':
        start_x = np.array(config['start'])
        lr = config.get('lr', 0.01)
        
        next_x = gradient_ascent_acquisition(
            ensemble, start_x, bounds, n_steps=50, lr=lr
        )
        
        print(f"Strategy: NN GRADIENT ASCENT (lr={lr})")
        grad = ensemble.gradient_at(start_x)
        print(f"Initial gradient: {['%.4f' % g for g in grad]}")
        pred_mu, pred_std = ensemble.predict(next_x.reshape(1, -1))
        print(f"Predicted after gradient ascent: y = {pred_mu[0]:.4f}")
    
    # Store and format query
    query_str = format_query(next_x)
    week5_queries[func_id] = {'query': query_str, 'array': next_x}
    
    print(f"\n>>> Week 5 Query: {query_str}")
    
    # Save submission
    save_submission(func_id, query_str, module_name="Module 16 - Week 5")
    
    print("-" * 80)

In [ ]:
print("=" * 80)
print("WEEK 5 QUERY SUMMARY")
print("=" * 80)

for func_id in range(1, 9):
    query = week5_queries[func_id]['query']
    strategy = WEEK5_CONFIG[func_id]['strategy']
    print(f"\nFunction {func_id} ({strategy}):")
    print(f"  {query}")

print("\n" + "=" * 80)
print("All queries saved to submissions/submission_log.csv")

In [ ]:
# Final summary table
print("\n" + "=" * 80)
print("WEEK 5 STRATEGY SUMMARY")
print("=" * 80)
print("\n{:<8} {:<4} {:<20} {:<12} {:<12}".format(
    "Function", "Dim", "Strategy", "Week 4 y", "Best Ever"
))
print("-" * 60)

week4_results = {
    1: 0.196, 2: 0.667, 3: -0.085, 4: -24.52,
    5: 1289.34, 6: -0.717, 7: 2.396, 8: 9.877
}

for func_id in range(1, 9):
    dim = function_data[func_id]['dim']
    strategy = WEEK5_CONFIG[func_id]['strategy']
    w4_y = week4_results[func_id]
    best_y = function_data[func_id]['y_best']
    
    print("{:<8} {:<4} {:<20} {:<12.4f} {:<12.4f}".format(
        f"F{func_id}", f"{dim}D", strategy, w4_y, best_y
    ))

print("\n" + "=" * 80)

---

## Reflection: Module 16 Concepts Applied to BBO

### 1. How do AlexNet's architectural innovations relate to our surrogate design?

**AlexNet's Key Innovations (2012)**:
- ReLU activation (fast, avoids vanishing gradients)
- Dropout regularization (prevents overfitting)
- GPU training (enables larger models)
- Data augmentation (more effective use of limited data)

**Application to our NN Surrogates**:
- **ReLU**: We use ReLU in all hidden layers for efficient gradient flow
- **Dropout**: Essential with only 14-44 samples; we use adaptive dropout (higher for smaller datasets)
- **LayerNorm**: We use LayerNorm instead of BatchNorm (works with batch size 1 for single-point prediction)
- **Ensemble**: Like data augmentation, using 5 diverse models provides implicit regularization

### 2. How does the concept of feature hierarchies apply to BBO?

In image classification:
- Layer 1: Detects edges and textures
- Layer 2: Combines into shapes
- Layer 3+: Recognizes objects

**In our BBO surrogates**:
- **Input layer**: Raw function coordinates
- **Hidden layer 1**: Learns which dimensions correlate with output (e.g., x₂, x₃ for F5)
- **Hidden layer 2**: Combines dimension interactions
- **Output**: Predicts function value

Evidence: For Function 5, gradients showed x₂ and x₃ have strong positive effects, while x₀ and x₁ are nearly neutral. The network learned this hierarchy automatically.

### 3. How do PyTorch's design choices benefit our approach?

**Dynamic Computation Graphs**:
- We can compute gradients at any arbitrary point x using `backward()`
- Essential for gradient-based acquisition functions
- Would be harder with TensorFlow's static graph (pre-2.0)

**Autograd**:
- `requires_grad_(True)` lets us compute ∂y/∂x without manual differentiation
- The `gradient_at()` method leverages this directly

**Module API**:
- Clean separation between architecture (`SurrogateNN`) and training (`fit()`)
- Easy to create diverse ensemble members

### 4. What are the five building blocks in our surrogate model?

| Block | Implementation | Purpose |
|-------|----------------|--------|
| **Linear** | `nn.Linear(prev_dim, hidden_dim)` | Transform input dimensions |
| **Activation** | `nn.ReLU()` | Non-linear function approximation |
| **Normalization** | `nn.LayerNorm(hidden_dim)` | Stabilize training, works with batch=1 |
| **Regularization** | `nn.Dropout(0.1-0.3)` | Prevent overfitting on small data |
| **Loss** | `nn.MSELoss()` | Regression objective |

### 5. How do architectural trade-offs in deep learning parallel BBO strategy?

**Depth vs Width**:
- Deeper networks: More expressive but harder to train (vanishing gradients)
- Wider networks: More parameters per layer but may overfit
- **BBO parallel**: Exploration (deep search) vs Exploitation (wide local search)

**Regularization Strength**:
- Too much: Underfitting (model too conservative)
- Too little: Overfitting (model memorizes noise)
- **BBO parallel**: Too much exploration wastes queries; too much exploitation misses better regions

**Learning Rate**:
- Too high: Training diverges
- Too low: Training too slow
- **BBO parallel**: Perturbation scale - too large jumps miss optima, too small gets stuck

### 6. Week 5 strategy informed by Module 16

**Evidence-based fine-tuning**:
- Like fine-tuning a pre-trained model, we start from known good points
- Use small learning rates (perturbation scales) to avoid destroying learned knowledge
- Function 1's breakthrough is like finding a good initialization - now we fine-tune

**Ensemble diversity**:
- Different architectures in our ensemble = implicit regularization
- Uncertainty from disagreement guides exploration
- Similar to how ImageNet models benefit from ensemble predictions

---

## Part 2: Strategy Reflection Questions

### 1. How did Week 4 results inform your Week 5 strategy?

**Key Lessons**:

1. **Function 1 Breakthrough**: The grid search at [0.65, 0.65] found a non-zero value (0.196) after 4 weeks of near-zero results. This is the most important discovery of the project. Week 5 must exploit this finding.

2. **Exact Return Works**: Functions 2 and 6 showed that returning to the best known coordinates can yield even better results. F2 improved from 0.611 (initial best) to 0.667.

3. **Gradient Methods Over-corrected**: Functions 3, 5, and 8 all regressed when using NN gradient ascent. The gradients pointed in wrong directions. For Week 5, we return to exact coordinates for these functions.

4. **F4 Exploration Failed**: Moving to [0.75, 0.75, 0.75, 0.75] gave -24.52, proving the positive region is NOT in the high corner. The Week 1 point remains the only positive value.

### 2. Which functions benefited most from neural network surrogates?

**Clear Benefits**:
- **F7**: NN micro-perturbation improved from 2.29 to 2.396 (new best)
- **F1**: NN uncertainty helped identify the [0.65, 0.65] region as worth exploring

**Mixed Results**:
- **F3, F5, F8**: NN gradients pointed in suboptimal directions

**Key Insight**: NNs work best for small refinements around known good points, not for large jumps.

### 3. How does the "exploration vs exploitation" trade-off manifest in deep learning concepts?

| Deep Learning | BBO Equivalent |
|---------------|----------------|
| High learning rate | Large perturbations (exploration) |
| Low learning rate | Small perturbations (exploitation) |
| Random initialization | Initial sampling (exploration) |
| Fine-tuning | Local refinement (exploitation) |
| Dropout | Stochastic exploration |
| Early stopping | Stopping when no improvement |

### 4. What would you do differently with more queries?

With unlimited queries:
1. **F1**: Dense grid search around [0.65, 0.65] to map the peak
2. **F4**: Systematic exploration of all corners to find the positive region's extent
3. **F5**: Latin Hypercube Sampling to better characterize the response surface
4. **All**: Bayesian optimization with Thompson Sampling for principled exploration

### 5. Final confidence levels for Week 5 queries

| Function | Confidence | Rationale |
|----------|------------|----------|
| F1 | **High** | Exploiting proven success at [0.65, 0.65] |
| F2 | **High** | Micro-perturbing W4's new best (0.667) |
| F3 | **Medium** | Returning to initial best, should be safe |
| F4 | **High** | Returning to only positive value |
| F5 | **High** | Returning to W1's proven best (1618.5) |
| F6 | **Medium** | Micro-perturbing near-optimal point |
| F7 | **Medium** | Gradient refinement from new best |
| F8 | **High** | Returning to W3's proven best (9.915) |

---

---

## Part 3: Critical Reflection on Week 5 Strategy

### 1. How did the ideas of hierarchical feature learning influence the way you thought about structuring or refining your optimisation strategy this round?

Hierarchical feature learning fundamentally changed how I approached the BBO problem this week. In deep learning, early layers learn simple features (edges, textures) that combine into increasingly complex representations. I applied this thinking to my optimization strategy in three ways:

**Layered Strategy Design**:
- **Layer 1 (Data)**: Raw function evaluations → identify which dimensions correlate with output
- **Layer 2 (Patterns)**: Combine dimension insights → recognize regional behavior (e.g., F5's high-corner optimum)
- **Layer 3 (Strategy)**: Synthesize into function-specific approaches → EXACT_RETURN vs GRADIENT_REFINE

**Dimension Importance Hierarchy**:
For Function 5, the NN learned that x₂ and x₃ are "high-level features" (strong gradients) while x₀ and x₁ are "low-level" (weak influence). This informed my decision to keep x₂, x₃ at boundary values while being flexible with x₀, x₁.

**Progressive Refinement**:
Like a CNN building from edges to objects, my strategy evolved from broad exploration (Week 1) → regional targeting (Week 3) → fine-tuning around discoveries (Week 5). The F1 breakthrough at [0.65, 0.65] is analogous to finding the right "feature" that the network can now refine.

---

### 2. What parallels do you see between breakthroughs like AlexNet/ImageNet and the incremental improvements in your capstone submissions?

**The AlexNet Moment**: In 2012, AlexNet didn't just win ImageNet—it shattered expectations by reducing error from 26% to 15%. This wasn't incremental; it was a paradigm shift enabled by three factors: **architecture** (deep CNNs), **data** (ImageNet scale), and **compute** (GPUs).

**Parallels in My BBO Journey**:

| AlexNet Breakthrough | BBO Capstone Parallel |
|---------------------|----------------------|
| ReLU enabling deep training | NN surrogates enabling gradient-guided search |
| Dropout preventing overfitting | Ensemble uncertainty preventing over-exploitation |
| ImageNet's scale | Accumulated 14+ samples per function |
| GPU acceleration | PyTorch's efficient autograd |

**My "AlexNet Moment"**: Function 1's breakthrough in Week 4 (0.196 after 4 weeks of zeros) parallels AlexNet's leap. It wasn't incremental improvement—it was finding the right region after systematic search. This single discovery changed my entire Week 5 strategy from exploration to exploitation.

**The Incremental Reality**: However, most of my improvements are incremental (F7: 2.29 → 2.396, a 4.6% gain). This mirrors post-AlexNet progress—VGGNet, ResNet, etc. made important but smaller gains. The lesson: breakthroughs are rare, but they redefine what's possible. My F1 breakthrough justifies continued investment in that region.

---

### 3. Did you encounter trade-offs between depth, complexity, and training efficiency similar to those in neural network design?

**Yes, directly analogous trade-offs emerged**:

**Depth vs Efficiency (Exploration vs Query Budget)**:
- **Deep exploration**: Systematic grid search of entire space would find global optimum but requires hundreds of queries
- **Shallow exploitation**: Local perturbation is efficient but may miss better regions
- **My compromise**: Adaptive depth—deeper search for functions with unclear optima (F4), shallow refinement for functions with known peaks (F5, F8)

**Complexity vs Overfitting (Model Trust vs Data Scarcity)**:
- **Complex NN**: Can model intricate response surfaces but may "hallucinate" patterns from 14 samples
- **Simple linear**: Won't overfit but misses non-linearities
- **My compromise**: Ensemble of diverse architectures—complexity averaged across models reduces overfitting while maintaining flexibility

**Training Efficiency Trade-off (Query Efficiency)**:
| Deep Learning | BBO Equivalent | My Week 5 Choice |
|---------------|----------------|------------------|
| Large batch size | Many parallel queries | Not available (1 query/week) |
| Learning rate scheduling | Adaptive perturbation scale | Start small (0.01-0.03), reduce over weeks |
| Early stopping | Stop exploring when confident | EXACT_RETURN for F4, F5, F8 |

**Critical Trade-off Decision**: For F4, I chose safety over potential gain. The NN predicted -28.5 at [0.75, 0.75, 0.75, 0.75], which proved correct (-24.52). Rather than risk another bad query, I'm returning to the only positive point—sacrificing exploration depth for exploitation efficiency.

---

### 4. Which neural network building blocks helped you think differently about how your model learns from accumulated data?

**Gradients (∂L/∂w → ∂y/∂x)**:
The most transformative concept. In neural networks, gradients tell us how to update weights. I applied this directly: **gradients of the surrogate tell us how to update query positions**. The `gradient_at()` method computes ∂y/∂x, revealing which direction improves the predicted output.

For F7, the gradient at the Week 4 best point showed:
```
[−0.15, +0.02, −0.08, +0.03, +0.05, −0.04]
```
This told me: decrease x₀, x₂; increase x₁, x₃, x₄. The Week 5 query follows this direction with conservative step size.

**Loss Functions (MSE → Acquisition Functions)**:
In NNs, the loss function defines what "good" means. I realized acquisition functions (UCB, EI) are analogous—they define what makes a query "good." UCB = μ + κσ balances predicted value (exploitation) with uncertainty (exploration), just as regularized loss balances fit and generalization.

**Weight Initialization → Starting Point Selection**:
Bad initialization can doom NN training. Similarly, choosing the right starting point for gradient ascent is critical. For F7, starting from the Week 4 best (2.396) rather than a random point leverages accumulated knowledge—like transfer learning.

**Activations (ReLU → Response Surface Shape)**:
ReLU creates piecewise-linear decision boundaries. This made me realize my surrogate learns a piecewise approximation of the true function. The "kinks" in the learned surface might not match the true function's shape, explaining why gradient methods over-corrected for F3, F5, F8.

---

### 5. Would your current optimisation approach be closer to rapid prototyping/flexibility or structured/production-ready design?

**Firmly in the "Rapid Prototyping" Paradigm** (PyTorch-style):

| Characteristic | PyTorch (Research) | TensorFlow (Production) | My BBO Approach |
|----------------|-------------------|------------------------|-----------------|
| **Iteration speed** | Fast experimentation | Careful deployment | Weekly pivots based on results |
| **Flexibility** | Change architecture mid-training | Static graph optimization | Function-specific strategies |
| **Debugging** | Easy inspection | Harder to trace | Direct visualization of NN predictions |
| **Reproducibility** | Set seeds, hope for best | Deterministic by design | Seeds set, but ensemble variance exists |

**Why Rapid Prototyping Fits**:

1. **Limited Data**: With only 14 samples per 2D function, I can't do production-style hyperparameter tuning. I need flexible architectures that adapt to data size.

2. **Weekly Feedback**: Each week's results inform next week's strategy. This is research-style iteration, not deployment.

3. **Uncertainty is Feature**: In production, you want confident predictions. Here, uncertainty (ensemble disagreement) actively guides exploration.

4. **Dynamic Strategy**: My approach changes per-function and per-week. F1 went from ADAPTIVE_GRID (Week 4) to EXPLOIT_DISCOVERY (Week 5). A production system would have fixed pipelines.

**What Would Make It Production-Ready**:
- Fixed architecture per function class (2D, 4D, 8D)
- Automated hyperparameter selection
- Guaranteed convergence bounds
- Deployment-ready query generation API

For now, flexibility > structure because the problem is still being understood.

---

### 6. How might reflecting on real-world deep learning use cases (like Giovanni Liotta's sports applications) inform how you benchmark success?

**Giovanni Liotta's Sports Analytics Insights**:
In the interview, Liotta discussed how deep learning transforms sports through:
- **Player tracking**: Real-time position data → tactical insights
- **Performance prediction**: Historical data → injury risk, peak performance timing
- **Strategy optimization**: Game theory + ML → optimal play calling

**Parallels to BBO Benchmarking**:

| Sports Analytics | BBO Capstone |
|------------------|--------------|
| Win/loss is ultimate metric | Best function value is ultimate metric |
| But process metrics matter (possession %, shot accuracy) | But learning metrics matter (R², uncertainty reduction) |
| Single game variance is high | Single query variance is high |
| Season-long trends reveal truth | Multi-week trends reveal truth |

**Revised Success Benchmarks**:

1. **Outcome Metrics** (like wins):
   - Primary: Did we beat or match best-ever value?
   - F1: 0.196 → 🎯 (new best)
   - F4: 0.600 target → need to recover

2. **Process Metrics** (like possession %):
   - Surrogate R²: Are we learning the function well?
   - Uncertainty reduction: Are we reducing epistemic uncertainty?
   - Gradient reliability: Do gradients point toward improvement?

3. **Learning Metrics** (like player development):
   - Did this week's result teach us something?
   - F4's -24.52 was a "loss" but taught us the high corner is bad—valuable information.

**Key Insight from Sports Analytics**:
Liotta emphasized that **short-term results can be noisy, but long-term strategy wins championships**. Applied to BBO: Week 4's F4 failure (-24.52) isn't just a bad result—it's information that narrows the search space. A sports team doesn't abandon a strategy after one loss; similarly, I shouldn't abandon exploration entirely after one bad query.

**Benchmark Evolution**:
- Weeks 1-2: Exploration success (coverage of space)
- Weeks 3-4: Exploitation success (improvement over best)
- Week 5: **Consolidation success** (protecting gains while seeking marginal improvements)

This is like a sports season: early games for experimentation, mid-season for building leads, late season for protecting position while staying competitive.

---

### Summary: Key Takeaways from Module 16 Reflection

1. **Hierarchical thinking** transformed my strategy from flat search to layered refinement
2. **AlexNet's breakthrough** parallels my F1 discovery—both enable new strategic possibilities
3. **Depth vs efficiency trade-offs** directly map to exploration vs exploitation decisions
4. **Gradients are the key insight**—they tell us how to improve, not just what's good
5. **Rapid prototyping mindset** is appropriate for research-phase optimization
6. **Sports analytics benchmarking** reminds us that process metrics matter alongside outcomes

The neural network isn't just a surrogate model—it's a **thinking framework** for approaching optimization.